In [2]:
import numpy as np
import json
import os
import copy
import pickle

import mesh_sampling
import trimesh
from shape_data import ShapeData

from autoencoder_dataset import cached_autoencoder_dataset, autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from train_funcs import train_autoencoder_dataloader
from test_funcs import test_autoencoder_dataloader


import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'trimesh' # 'mpi-mesh', 'trimesh'
root_dir = '/home/jingwang/Data/data/'

dataset = 'FaceWarehouse'
name = 'neutral'

torch.backends.cudnn.benchmark = True


GPU = True
os.environ['CUDA_VISIBLE_DEVICES']="3"
device_idx = 0
print(torch.cuda.get_device_name(device_idx))

ModuleNotFoundError: No module named 'shape_data'

In [2]:
args = {}

generative_model = 'pca_autoencoder'

reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed',name),
        'results_folder':  os.path.join(root_dir, dataset,'results/fully_connected_'+ generative_model),
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size': 8, 'num_epochs':3000, 'eval_frequency':200, 'num_workers': 40,
        'nz': 16,
        
        'lr':1e-3, 
        'regularization': 5e-5,         
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,  
        'resume': False,
        'reference_mesh_file': reference_mesh_file,
        'mode':'test', 'shuffle': False, 'nVal': 100, 'normalization': True,
        'write_mesh': True,
        'lambda_var': 0.5,
        'worst_face_num': 20}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

downsample_mesh_path = os.path.join(args['results_folder'],'downsample_mesh', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)


worst_mesh_path = os.path.join(args['results_folder'],'worst_test', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)

In [3]:
np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train.npy', 
                          test_file=args['data']+'/test.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'], 
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy', 
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = False)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]

specified material ()  not loaded!


Loading data .. 


In [4]:
torch.manual_seed(args['seed'])

if GPU:
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


In [5]:
# Building model, optimizer, and loss function

dataset_train = cached_autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                           shapedata = shapedata,
                                           normalization = args['normalization'],device=device)

dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                     shuffle = args['shuffle'])

dataset_val = cached_autoencoder_dataset(root_dir = args['data'], points_dataset = 'val', 
                                         shapedata = shapedata,
                                         normalization = args['normalization'],device=device)

dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                     shuffle = False)


dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          shapedata = shapedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'],  pin_memory=True)

if 'pca_autoencoder' == args['generative_model']:
    import torch.nn as nn
    class PcaAutoEncoder(nn.Module):
        def __init__(self, nz):
            super().__init__()
            self.nz = nz
            self.en_fc = nn.Linear(11511*3, nz)
            self.de_fc = nn.Linear(nz, 11511*3)
            self._only_encode = False
            self._only_decode = False
        
        def only_encode(self, status):
            self._only_encode = status
        
        def only_decode(self, status):
            self._only_decode = status
        
        def encode(self, x):
            z = self.en_fc(x)
            return z
        
        def decode(self, z):
            x = self.de_fc(z)
            return x
        
        def forward(self, x):
            if self._only_decode:
                x = self.decode(x)
                x = x.reshape(x.shape[0], -1, 3)
                return x
            x = x.reshape(x.shape[0], -1)
            z = self.encode(x)
            if self._only_encode:
                return z
            x = self.decode(z)
            x = x.reshape(x.shape[0], -1, 3)
            return x
        
    model = PcaAutoEncoder(nz=args['nz']).to(device)
    
optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None

def loss_l2(outputs, targets):
    L = torch.sqrt(torch.mean((outputs - targets)**2))
    return L
    
if args['loss']=='l1':
    def loss_l1(outputs, targets):
        L = torch.abs(outputs - targets).mean()
        return L 
    loss_fn = loss_l1
elif arg['loss']=='l1_var':
    lambda_var = args['lambda_var']
    def variational_loss(tx,tx_hat):
        x,mu,logvar = tx
        l1_loss = torch.mean(torch.abs(x-tx))
        var_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return l1_loss + lambda_var * var_loss
    loss_fn = variational_loss



In [6]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)
# print(M[4].v.shape)

Total number of parameters is: 1139605
PcaAutoEncoder(
  (en_fc): Linear(in_features=34533, out_features=16, bias=True)
  (de_fc): Linear(in_features=16, out_features=34533, bias=True)
)


In [7]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'],'checkpoints', args['name'] +'_params.json'),'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)
        
    if args['resume']:
            print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file'])))
            checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
            start_epoch = checkpoint_dict['epoch'] + 1
            model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
            optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
            print('Resuming from epoch %s'%(str(start_epoch)))     
    else:
        start_epoch = 0
        
    if args['generative_model'] in ['autoencoder', 'simple_autoencoder', 'pca_autoencoder']:
        train_autoencoder_dataloader(dataloader_train, dataloader_val,
                          device, model, optim, loss_fn,
                          bsize = args['batch_size'],
                          start_epoch = start_epoch,
                          n_epochs = args['num_epochs'],
                          eval_freq = args['eval_frequency'],
                          scheduler = scheduler,
                          writer = writer,
                          save_recons=True,
                          shapedata=shapedata,
                          metadata_dir=checkpoint_path, samples_dir=samples_path,
                          checkpoint_path = args['checkpoint_file'])

In [17]:
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_simple_autoencoder/latent_128/checkpoints/align_pose/checkpoint.pth.tar


100%|██████████| 21/21 [00:02<00:00,  8.53it/s]


('loss for ', 612, tensor(4.3394, device='cuda:0'))
('loss for ', 602, tensor(4.3528, device='cuda:0'))
('loss for ', 194, tensor(4.3574, device='cuda:0'))
('loss for ', 560, tensor(4.3619, device='cuda:0'))
('loss for ', 390, tensor(4.4021, device='cuda:0'))
('loss for ', 402, tensor(4.3611, device='cuda:0'))
('loss for ', 537, tensor(4.4436, device='cuda:0'))
('loss for ', 625, tensor(4.3799, device='cuda:0'))
('loss for ', 8, tensor(4.4154, device='cuda:0'))
('loss for ', 628, tensor(4.5263, device='cuda:0'))
('loss for ', 559, tensor(4.6070, device='cuda:0'))
('loss for ', 626, tensor(4.6145, device='cuda:0'))
('loss for ', 370, tensor(4.4514, device='cuda:0'))
('loss for ', 125, tensor(4.5456, device='cuda:0'))
('loss for ', 389, tensor(4.4977, device='cuda:0'))
('loss for ', 364, tensor(4.5176, device='cuda:0'))
('loss for ', 169, tensor(4.5499, device='cuda:0'))
('loss for ', 499, tensor(4.4825, device='cuda:0'))
('loss for ', 341, tensor(4.4610, device='cuda:0'))
('loss for ', 

In [7]:
if args['mode'] == 'test': # test with train set
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_train, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/fully_connected_simple_autoencoder/latent_128/checkpoints/align_pose/checkpoint.pth.tar


100%|██████████| 290/290 [00:18<00:00, 15.86it/s]


('loss for ', 4407, tensor(0.7333, device='cuda:0'))
('loss for ', 7990, tensor(0.7386, device='cuda:0'))
('loss for ', 4652, tensor(0.7499, device='cuda:0'))
('loss for ', 4229, tensor(0.7571, device='cuda:0'))
('loss for ', 9272, tensor(0.8053, device='cuda:0'))
('loss for ', 5009, tensor(0.7569, device='cuda:0'))
('loss for ', 5751, tensor(0.7895, device='cuda:0'))
('loss for ', 3968, tensor(0.7604, device='cuda:0'))
('loss for ', 8612, tensor(0.7764, device='cuda:0'))
('loss for ', 4689, tensor(0.8434, device='cuda:0'))
('loss for ', 2876, tensor(0.8575, device='cuda:0'))
('loss for ', 8032, tensor(0.7779, device='cuda:0'))
('loss for ', 2945, tensor(0.7627, device='cuda:0'))
('loss for ', 6959, tensor(0.8138, device='cuda:0'))
('loss for ', 1443, tensor(0.9869, device='cuda:0'))
('loss for ', 988, tensor(0.8157, device='cuda:0'))
('loss for ', 2040, tensor(0.8673, device='cuda:0'))
('loss for ', 2952, tensor(0.7869, device='cuda:0'))
('loss for ', 8520, tensor(0.8912, device='cuda

In [55]:
if args['mode'] == 'test': # test with val set
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_autoencoder_dataloader(device, model, dataloader_val, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

100%|██████████| 2/2 [00:00<00:00, 186.53it/s]

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/fully_connected_pca_autoencoder/latent_16/checkpoints/neutral/checkpoint.pth.tar
loss for  7 tensor(1.1442, device='cuda:0')
loss for  5 tensor(1.4197, device='cuda:0')
loss for  0 tensor(1.4422, device='cuda:0')
loss for  2 tensor(1.5218, device='cuda:0')
loss for  3 tensor(1.6134, device='cuda:0')
loss for  2 tensor(1.7671, device='cuda:0')
loss for  6 tensor(1.6526, device='cuda:0')
loss for  1 tensor(1.6962, device='cuda:0')
loss for  3 tensor(1.6305, device='cuda:0')
loss for  4 tensor(1.6477, device='cuda:0')
autoencoder: normalized loss 0.24331627786159515
autoencoder: euclidean distance in mm= 1.5535428524017334


In [10]:
# from optimize_to_get_result import optimize_to_get_result

import torch
import torch.optim as optim
import torch.nn as nn


def optimize_to_get_result(model, loss_fun, z_dim, device, targets, n_iter=1, output_loss=True):
    model.eval()
    
#     return torch.empty(0), model(targets)
    
    model.only_encode(True)
    z = model(targets)
    model.only_encode(False)
    
    model.only_decode(True)
    z_param = nn.Parameter(z)
    optimizer = optim.LBFGS(params=[z_param])
    for it in range(n_iter):
        def closure():
            optimizer.zero_grad()
            outputs = model(z_param)
            outputs = outputs.reshape(targets.shape) # decode, don't know shape
            loss = loss_fun(outputs, targets)
            loss.backward()
            if output_loss:
                print('loss ', loss.item())
            return loss
        optimizer.step(closure)
    outputs = model(z_param)
    outputs = outputs.reshape(targets.shape)
    
    model.only_decode(False)
    return z_param.data, outputs

def work_optimize():

    test_dataset = dataset_val
    # test_ids = [402,560, 50,  537, 390,
    #             364, 169, 616, 341, 590,
    #             499, 389,125, 370, 614,
    #             600, 194, 8, 653, 559]
#     test_ids = [602,194,560,390,402,537,625,8, 628,559,626,370,125,389,364,169,499,341,50]
    test_ids = list(range(10))
    inputs = torch.cat([test_dataset[idx]['points'].to(device).unsqueeze(0) for idx in test_ids])
    inputs = inputs.to(device)

    print(inputs.shape) # fixme

    shapedata_mean = torch.Tensor(shapedata.mean).to(device)
    shapedata_std = torch.Tensor(shapedata.std).to(device)

    z, outputs = optimize_to_get_result(model, loss_fn, args['nz'], device,
                          inputs)
    from pprint import pprint
    pprint(z)

    outputs = outputs[:,:-1]
    inputs = inputs[:,:-1]

    old_outputs = outputs
    old_inputs = inputs

    outputs = outputs * shapedata_std + shapedata_mean
    inputs = inputs * shapedata_std + shapedata_mean

    per_l2_loss = torch.sqrt(torch.sum((outputs - inputs)**2, dim=2))

    for i in range(len(test_ids)):
        print(test_ids[i], torch.mean(per_l2_loss[i]))

    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_input'),old_inputs.detach().cpu().numpy(),test_ids)
    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_output'),old_outputs.detach().cpu().numpy(),test_ids)

work_optimize()

torch.Size([10, 11511, 3])
loss  0.24329513311386108
loss  0.24329493939876556
loss  0.24329471588134766
loss  0.24329450726509094
loss  0.24329429864883423
loss  0.2432941049337387
loss  0.2432938665151596
loss  0.24329368770122528
loss  0.24329344928264618
loss  0.24329324066638947
loss  0.24329303205013275
loss  0.24329283833503723
loss  0.24329259991645813
loss  0.2432924211025238
loss  0.2432921975851059
loss  0.243291974067688
loss  0.24329176545143127
loss  0.24329157173633575
loss  0.24329133331775665
loss  0.24329115450382233
tensor([[-2.1633e+01,  2.0553e+01,  4.9337e+01,  1.6851e+00,  2.3712e+01,
         -3.7888e+01, -1.4641e+01, -3.7942e+01,  3.1905e+01, -2.4031e+01,
          1.8875e+01, -3.5214e+01, -2.7131e+01,  3.2612e+01,  7.3559e+00,
         -9.1403e-01],
        [ 5.0887e+00, -4.2406e+00, -1.8859e+01,  4.0508e+01,  3.1154e+01,
         -6.6240e+01, -2.3830e+01, -6.9777e+01,  4.2486e+01,  1.3714e+00,
          3.7937e+01,  2.1957e+00,  4.2534e+01,  3.6115e+00, -3.74

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=args['nz'])
X = np.array([dataset_train[i]['points'].cpu().numpy() for i in range(len(dataset_train))])

In [12]:
X = X.reshape(X.shape[0], -1)
W = pca.fit_transform(X.T)

In [13]:
W.shape

(34533, 16)

In [14]:
X.shape

(140, 34533)

In [15]:
X_val = np.array([dataset_val[i]['points'].cpu().numpy() for i in range(len(dataset_val))])

In [16]:
X_val.shape

(10, 11511, 3)

In [17]:
X_mean = np.mean(X, axis=0)

In [18]:
X_val = X_val.reshape(X_val.shape[0], -1).T
X_val.shape

(34533, 10)

In [19]:
x = np.linalg.lstsq(W,X_val-X_mean.reshape(-1,1),rcond=None)[0]

In [20]:
x.shape

(16, 10)

In [21]:
pca_recon = W@x+X_mean.reshape(-1,1)

In [22]:
pca_recon.shape

(34533, 10)

In [46]:
shapedata.save_meshes(os.path.join(worst_mesh_path,'pca'),pca_recon.T, list(range(10)))

0

In [42]:
shapedata.save_meshes(os.path.join(worst_mesh_path,'val_input'),X_val.T,list(range(10)))

0

In [24]:
X_val.shape

(34533, 10)

In [28]:
shapedata.std.shape

(11510, 3)

In [29]:
shapedata.mean.shape

(11510, 3)

In [30]:
X_val = X_val[:-3]
pca_recon = pca_recon[:-3]

In [47]:
X_val_denorm = X_val * shapedata.std.reshape(-1,1) + shapedata.mean.reshape(-1,1)
pca_denorm = pca_recon * shapedata.std.reshape(-1,1) + shapedata.mean.reshape(-1,1)

In [33]:
X_val_denorm.shape

(34530, 10)

In [48]:
pca_denorm.shape

(34530, 10)

In [49]:
residual = (X_val_denorm-pca_denorm)**2

In [50]:
residual = residual.reshape(-1,3,10)

In [51]:
residual = np.sqrt(np.sum(residual,axis=1))

In [52]:
residual.shape

(11510, 10)

In [54]:
np.mean(residual)*100

1.7689099535346031

In [34]:
pca_recon.shape

(34530, 10)

In [41]:
np.mean(np.sqrt((pca_recon-X_val)**2))

0.27528316

In [38]:
pca_recon[0,0]

0.899289

In [39]:
X_val_denorm[0,0]

-0.62963533

In [40]:
X_val[0,0]

0.9558933

In [84]:
pca_recon = pca_recon[:-3].T

In [86]:
pca_recon = pca_recon.reshape(pca_recon.shape[0],-1,3)

In [87]:
pca_recon.shape

(10, 11510, 3)

In [92]:
X_val.shape

(34533, 10)

In [93]:
X_val_denorm.shape

(10, 11510, 3)

In [94]:
np.mean(X_val_denorm)

0.057275042

In [95]:
np.mean(pca_recon)

-0.0565793

In [99]:
residual = (X_val-pca_recon)

0.27528813

In conclusion, when latent is 16, PCA's reconstrution error is 1.76mm, while autoencoder's recons error is 1.55mm

1.55mm doesn't need to optimize